# Hand-Made Filters and Convolutions
## Understanding CNN Building Blocks Through Manual Implementation

### Learning Objectives:
- Implement convolution operation from scratch
- Understand how different filters detect different features
- Create custom filters for edge, corner, and other feature detection
- Visualize the effects of various convolutional filters

### Structure:
1. Basic convolution implementation
2. Classic edge detection filters (Sobel, Prewitt)
3. Diagonal edge detection
4. **Exercise:** Corner detection filters
5. **Exercise:** Custom feature detection

## Setup and Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import requests
from io import BytesIO
from typing import Tuple

# Set up matplotlib for better display
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['image.cmap'] = 'gray'

## Create Test Images

We'll create simple geometric shapes to clearly see the effects of our filters.

In [ ]:
def create_test_images():
    """Create simple test images with various features."""
    images = {}

    # 1. Vertical and horizontal lines
    img_lines = np.zeros((100, 100))
    img_lines[40:60, 20] = 255  # Vertical line
    img_lines[40:60, 80] = 255  # Vertical line
    img_lines[20, 30:70] = 255  # Horizontal line
    img_lines[80, 30:70] = 255  # Horizontal line
    images['lines'] = img_lines

    # 2. Square
    img_square = np.zeros((100, 100))
    img_square[30:70, 30:70] = 255
    images['square'] = img_square

    # 3. Diagonal lines
    img_diag = np.zeros((100, 100))
    for i in range(40):
        img_diag[30+i, 30+i] = 255  # Diagonal \
        img_diag[30+i, 70-i] = 255  # Diagonal /
    images['diagonals'] = img_diag

    # 4. Corners (L-shapes)
    img_corners = np.zeros((100, 100))
    # Top-left corner
    img_corners[20:30, 20] = 255
    img_corners[20, 20:30] = 255
    # Top-right corner
    img_corners[20:30, 70] = 255
    img_corners[20, 70:80] = 255
    # Bottom-left corner
    img_corners[70:80, 20] = 255
    img_corners[80, 20:30] = 255
    # Bottom-right corner
    img_corners[70:80, 70] = 255
    img_corners[80, 70:80] = 255
    images['corners'] = img_corners

    # 5. Circle
    img_circle = np.zeros((100, 100))
    center = (50, 50)
    radius = 25
    y, x = np.ogrid[:100, :100]
    mask = (x - center[0])**2 + (y - center[1])**2 <= radius**2
    img_circle[mask] = 255
    images['circle'] = img_circle

    return images

# Create test images
test_images = create_test_images()

# Display all test images
fig, axes = plt.subplots(1, 5, figsize=(15, 3))
for ax, (name, img) in zip(axes, test_images.items()):
    ax.imshow(img, cmap='gray')
    ax.set_title(name.capitalize())
    ax.axis('off')
plt.tight_layout()
plt.show()

## Part 1: Implementing Convolution from Scratch

Let's implement the 2D convolution operation manually to understand exactly what's happening.

In [ ]:
def convolve2d(image: np.ndarray, kernel: np.ndarray, padding: str = 'valid') -> np.ndarray:
    """
    Perform 2D convolution operation.

    Args:
        image: Input image (2D array)
        kernel: Convolution kernel (2D array)
        padding: 'valid' (no padding) or 'same' (preserve size)

    Returns:
        Convolved image
    """
    # Get dimensions
    image_height, image_width = image.shape
    kernel_height, kernel_width = kernel.shape

    # Apply padding if needed
    if padding == 'same':
        pad_h = kernel_height // 2
        pad_w = kernel_width // 2
        image = np.pad(image, ((pad_h, pad_h), (pad_w, pad_w)), mode='constant')
        image_height, image_width = image.shape

    # Calculate output dimensions
    output_height = image_height - kernel_height + 1
    output_width = image_width - kernel_width + 1

    # Initialize output
    output = np.zeros((output_height, output_width))

    # Perform convolution
    for i in range(output_height):
        for j in range(output_width):
            # Extract region
            region = image[i:i+kernel_height, j:j+kernel_width]
            # Element-wise multiplication and sum
            output[i, j] = np.sum(region * kernel)

    return output

In [ ]:
# Test with a simple averaging kernel
averaging_kernel = np.ones((3, 3)) / 9
print("Averaging kernel:")
print(averaging_kernel)

In [ ]:
# Apply to a diagonals image with size-preserving padding
print(f"Input image dimensions: {test_images['diagonals'].shape}")
result = convolve2d(test_images['diagonals'], averaging_kernel, padding='same')
print(f"Output image dimensions: {result.shape}")

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))
ax1.imshow(test_images['diagonals'], cmap='gray')
ax1.set_title('Original Diagonals')
ax1.axis('off')
ax2.imshow(result, cmap='gray')
ax2.set_title('After Averaging Filter')
ax2.axis('off')
plt.show()

In [ ]:
# Apply to square image with no padding
print(f"Input image dimensions: {test_images['square'].shape}")
result = convolve2d(test_images['square'], averaging_kernel, padding='valid')
print(f"Output image dimensions: {result.shape}")

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))
ax1.imshow(test_images['square'], cmap='gray')
ax1.set_title('Original Square')
ax1.axis('off')
ax2.imshow(result, cmap='gray')
ax2.set_title('After Averaging Filter')
ax2.axis('off')
plt.show()

## Part 2: Classic Edge Detection Filters

### Sobel Operators
The Sobel operator uses two 3×3 kernels to detect edges in horizontal and vertical directions.

In [ ]:
# Define Sobel operators
sobel_x = np.array([[-1, 0, 1],
                    [-2, 0, 2],
                    [-1, 0, 1]], dtype=np.float32)

sobel_y = np.array([[-1, -2, -1],
                    [ 0,  0,  0],
                    [ 1,  2,  1]], dtype=np.float32)

print("Sobel X (Vertical Edge Detector):")
print(sobel_x)
print("\nSobel Y (Horizontal Edge Detector):")
print(sobel_y)

In [ ]:
def apply_sobel_filters(image: np.ndarray) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """
    Apply Sobel filters to detect edges.

    Returns:
        - Gradient in X direction
        - Gradient in Y direction
        - Gradient magnitude
    """
    # Apply Sobel filters
    grad_x = convolve2d(image, sobel_x, padding='same')
    grad_y = convolve2d(image, sobel_y, padding='same')

    # Compute gradient magnitude
    grad_magnitude = np.sqrt(grad_x**2 + grad_y**2)

    return grad_x, grad_y, grad_magnitude

# Apply to different test images
for img_name in ['lines', 'square', 'circle']:
    img = test_images[img_name]
    grad_x, grad_y, grad_mag = apply_sobel_filters(img)

    fig, axes = plt.subplots(1, 4, figsize=(16, 4))

    axes[0].imshow(img, cmap='gray')
    axes[0].set_title(f'Original {img_name}')

    axes[1].imshow(grad_x, cmap='RdBu_r')
    axes[1].set_title('Sobel X (Vertical Edges)')

    axes[2].imshow(grad_y, cmap='RdBu_r')
    axes[2].set_title('Sobel Y (Horizontal Edges)')

    axes[3].imshow(grad_mag, cmap='gray')
    axes[3].set_title('Gradient Magnitude')

    for ax in axes:
        ax.axis('off')

    plt.tight_layout()
    plt.show()

### Prewitt Operators
Similar to Sobel but with equal weights.

In [ ]:
# Define Prewitt operators
prewitt_x = np.array([[-1, 0, 1],
                      [-1, 0, 1],
                      [-1, 0, 1]], dtype=np.float32)

prewitt_y = np.array([[-1, -1, -1],
                      [ 0,  0,  0],
                      [ 1,  1,  1]], dtype=np.float32)

# Compare Sobel vs Prewitt on the square image
img = test_images['square']

sobel_result = convolve2d(img, sobel_x, padding='same')
prewitt_result = convolve2d(img, prewitt_x, padding='same')

fig, axes = plt.subplots(1, 3, figsize=(12, 4))
axes[0].imshow(img, cmap='gray')
axes[0].set_title('Original')
axes[1].imshow(sobel_result, cmap='RdBu_r')
axes[1].set_title('Sobel X')
axes[2].imshow(prewitt_result, cmap='RdBu_r')
axes[2].set_title('Prewitt X')

for ax in axes:
    ax.axis('off')
plt.tight_layout()
plt.show()

## Part 3: Question 1 - Diagonal Edge Detection

We can create filters to detect diagonal edges by rotating the Sobel pattern.

In [ ]:
# Question 1: Define diagonal edge detection filters
# ==================================================
# Define the following attributes as np.array objects:
# sobel_diag1 = ?
# sobel_diag2 = ?



# Test diagonal filters
img = test_images['diagonals']

diag1_result = convolve2d(img, sobel_diag1, padding='same')
diag2_result = convolve2d(img, sobel_diag2, padding='same')

fig, axes = plt.subplots(1, 4, figsize=(16, 4))
axes[0].imshow(img, cmap='gray')
axes[0].set_title('Original Diagonals')
axes[1].imshow(diag1_result, cmap='RdBu_r')
axes[1].set_title('Diagonal \\ Detection')
axes[2].imshow(diag2_result, cmap='RdBu_r')
axes[2].set_title('Diagonal / Detection')
axes[3].imshow(np.abs(diag1_result) + np.abs(diag2_result), cmap='gray')
axes[3].set_title('Both Diagonals')

for ax in axes:
    ax.axis('off')
plt.tight_layout()
plt.show()

## Part 4: Questions 2 & 3 - Corner Detection

**Task:** Create filters that can detect different types of corners (L-shapes).

Think about what pattern would indicate a corner:
- Top-left corner: High values in bottom and right, low elsewhere
- Top-right corner: High values in bottom and left, low elsewhere
- etc.

Create four 3×3 filters to detect the four types of corners.

In [ ]:
# Question 2: Define corner detection filters
# ===========================================
# Hint: Think about the pattern of positive and negative values
# that would respond strongly to an L-shaped corner
#
# Define the following attributes as np.array objects:
# corner_top_left = ?
# corner_top_right = ?
# corner_bottom_left = ?
# corner_bottom_right = ?

In [ ]:
def visualize_corner_detection(img):
    fig, axes = plt.subplots(2, 3, figsize=(12, 8))

    # Original image
    axes[0, 0].imshow(img, cmap='gray')
    axes[0, 0].set_title('Original Corners')

    # Apply each corner filter
    corner_filters = [
        (corner_top_left, 'Top-Left'),
        (corner_top_right, 'Top-Right'),
        (corner_bottom_left, 'Bottom-Left'),
        (corner_bottom_right, 'Bottom-Right')
    ]

    for idx, (filter_kernel, name) in enumerate(corner_filters):
        result = convolve2d(img, filter_kernel, padding='same')
        row = (idx + 1) // 3
        col = (idx + 1) % 3
        axes[row, col].imshow(result, cmap='RdBu_r')
        axes[row, col].set_title(f'{name} Corner Detection')

    # Combined result
    all_corners = np.zeros_like(img, dtype=np.float32)
    for filter_kernel, _ in corner_filters:
        result = convolve2d(img, filter_kernel, padding='same')
        all_corners = np.maximum(all_corners, result)

    axes[1, 2].imshow(all_corners, cmap='hot')
    axes[1, 2].set_title('All Corners Combined')

    for ax in axes.flat:
        ax.axis('off')

    plt.tight_layout()
    plt.show()

In [ ]:
# Test corner detection
img = test_images['corners']
visualize_corner_detection(img)

### Do your filters work well for the square image?

Remember, only the edges or corners should be highlighted, not the inside of the square!

In [ ]:
img = test_images['square']
visualize_corner_detection(img)

**If not, adapt your filters so they do!**

In [ ]:
# Question 3: Full Corner Detection
# =================================
#
# Re-define the filters to work for full corners:
# corner_top_left = ?
# corner_top_right = ?
# corner_bottom_left = ?
# corner_bottom_right = ?


# Test corner detection
img = test_images['square']
visualize_corner_detection(img)

## Part 5: Question 4 - Custom Feature Detection

**Task:** Design a filter that detects circular edges (like the outline of a circle).

Hint: Think about what makes a circular edge different from straight edges.
You might want to use a larger kernel (5×5) for better circular pattern detection.

In [ ]:
# Question 4: Create a filter for circular edge detection
# =======================================================
# This is more challenging - think about radial gradients
# Define the following attribute as an np.array object:
# log_filter = ?
#
# HINT: If you get stuck, search for Laplacian of Gaussian
# (but try figuring it out alone first!).



# Test circular edge detection
img = test_images['circle']

# Apply filter
log_result = convolve2d(img, log_filter, padding='same')

# Also try edge detection first, then circular pattern
edges_x, edges_y, edges_mag = apply_sobel_filters(img)

fig, axes = plt.subplots(1, 3, figsize=(12, 8))

axes[0].imshow(img, cmap='gray')
axes[0].set_title('Original Circle')

axes[1].imshow(edges_mag, cmap='gray')
axes[1].set_title('Sobel Edge Magnitude')

axes[2].imshow(log_result, cmap='RdBu_r')
axes[2].set_title('Laplacian of Gaussian')

# Threshold to show detected circles
for ax in axes.flat:
    ax.axis('off')

plt.tight_layout()
plt.show()

## Part 6: Real Image Application

Let's apply our filters to a real image to see how they work in practice.

In [ ]:
# Load a real image
# Using a simple geometric image for clarity
def create_real_test_image():
    """Create a more complex test image with multiple features."""
    img = np.zeros((200, 200), dtype=np.float32)

    # Add various features
    # Rectangle
    img[50:100, 30:80] = 200

    # Circle
    center = (150, 60)
    y, x = np.ogrid[:200, :200]
    mask = (x - center[0])**2 + (y - center[1])**2 <= 30**2
    img[mask] = 150

    # Triangle
    for i in range(40):
        for j in range(i):
            img[140 + i, 100 + j - i//2] = 180

    # Add some noise
    noise = np.random.normal(0, 10, img.shape)
    img = img + noise
    img = np.clip(img, 0, 255)

    return img

In [ ]:
real_img = create_real_test_image()

In [ ]:
plt.figure(figsize=(6, 6))
plt.imshow(real_img, cmap='gray')
plt.title('Real Test Image')
plt.axis('off')
plt.show()

In [ ]:
# Apply various filters
filters = {
    'Sobel X': sobel_x,
    'Sobel Y': sobel_y,
    'Laplacian': np.array([[0, -1, 0], [-1, 4, -1], [0, -1, 0]]),
    'Sharpening': np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]]),
    'Gaussian Blur': np.ones((5, 5)) / 25
}

fig, axes = plt.subplots(2, 3, figsize=(12, 8))
axes = axes.flatten()

axes[0].imshow(real_img, cmap='gray')
axes[0].set_title('Original Image')
axes[0].axis('off')

for idx, (name, kernel) in enumerate(filters.items(), 1):
    result = convolve2d(real_img, kernel, padding='same')
    axes[idx].imshow(result, cmap='gray' if 'Blur' in name or 'Sharp' in name else 'RdBu_r')
    axes[idx].set_title(name)
    axes[idx].axis('off')

plt.tight_layout()
plt.show()

## Summary and Key Takeaways

1. **Convolution** is a fundamental operation that slides a kernel over an image
2. **Different kernels detect different features:**
   - Sobel/Prewitt: Edge detection
   - Laplacian: Second derivatives, blob detection
   - Custom kernels: Specific pattern detection

3. **Edge detection** forms the basis of many computer vision algorithms
4. **Feature hierarchies:** Simple features (edges) combine to form complex features

### Connection to CNNs:
- CNNs **learn** these kernels automatically from data
- Early CNN layers often learn edge detectors similar to Sobel
- Deeper layers learn more complex feature detectors
- The power of CNNs: discovering optimal features for the task

### Next Steps:
In the main hands-on session, we'll see how PyTorch implements these convolutions efficiently and how CNNs learn these filters automatically!

## Filter Visualization

Here is a function that visualizes what each filter is "looking for" by showing its positive and negative components separately.

In [ ]:
def visualize_filter(kernel, title="Filter"):
    """Visualize a filter kernel showing positive and negative parts."""
    fig, axes = plt.subplots(1, 3, figsize=(10, 3))

    # Full kernel
    im1 = axes[0].imshow(kernel, cmap='RdBu_r', vmin=-np.abs(kernel).max(), vmax=np.abs(kernel).max())
    axes[0].set_title(f'{title} - Full')
    plt.colorbar(im1, ax=axes[0], fraction=0.046)

    # Positive parts
    positive = np.maximum(kernel, 0)
    im2 = axes[1].imshow(positive, cmap='Reds')
    axes[1].set_title('Positive Weights')
    plt.colorbar(im2, ax=axes[1], fraction=0.046)

    # Negative parts
    negative = np.minimum(kernel, 0)
    im3 = axes[2].imshow(negative, cmap='Blues_r')
    axes[2].set_title('Negative Weights')
    plt.colorbar(im3, ax=axes[2], fraction=0.046)

    for ax in axes:
        ax.set_xticks(range(kernel.shape[1]))
        ax.set_yticks(range(kernel.shape[0]))
        ax.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

In [ ]:
# Visualize all our edge detection filters
visualize_filter(sobel_x, "Sobel X")
visualize_filter(sobel_y, "Sobel Y")
visualize_filter(sobel_diag1, "Diagonal 1")
visualize_filter(corner_top_left, "Corner Top-Left")